In [44]:
#性能测量
def meansure_performance(y_predict_lis):
    TP=0
    FP=0
    TN=0
    FN=0
    #测试样本数
    sample_num=x_test.shape[0]
    for i in range(sample_num):
        y_predict=y_predict_lis[i]
        if y_predict==1 and y_test[i]==1:
            TP+=1
        elif y_predict==-1 and y_test[i]==-1:
            TN+=1
        elif y_predict==1 and y_test[i]==-1:
    #false poistive
            FP+=1
        elif y_predict==-1 and y_test[i]==1:
    #false negative
            FN+=1 
    accuracy=(TP+TN)/(TP+FP+TN+FN)
    recall=TP/(TP+FN)
    specificity=TN/(TN+FP)
    precision=TP/(TP+FP)
    F_measuer=(2*precision*recall)/(precision+recall)
    return accuracy,recall,specificity,precision,F_measuer

In [45]:
from skfeature.function.similarity_based import reliefF
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_guizhou.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
x_test=fea[:170,:]
y_test=target[:170]
x_train=fea[170:,:]
y_train=target[170:]

In [46]:
techini_indicator=data_rfe.columns.tolist()[:-1]

# SVC

In [47]:
estimator_SVC=SVC(C=0.1, kernel='rbf',gamma=0.1)
estimator_SVC.fit(x_train,y_train)
SVC_result= estimator_SVC.predict(x_test)
performan_SVC=meansure_performance(SVC_result)
print(performan_SVC)

(0.5647058823529412, 0.9726027397260274, 0.25773195876288657, 0.4965034965034965, 0.6574074074074074)


 # CFS-SVM

In [48]:
CFS_selected_feature = CFS.cfs(x_train,y_train)
CFS_fea_sele=list(CFS_selected_feature)
CFS_fea_sele

[23, 6, 28, 39, 1, 2, 4, 5]

In [49]:
np.array(techini_indicator)[CFS_fea_sele],len(np.array(techini_indicator)[CFS_fea_sele])

(array(['CCI', 'CR', 'J', 'MI', 'TAPI', 'VMA', 'VSTD', 'VMACD'],
       dtype='<U8'),
 8)

In [50]:
estimator_SVM_CFS=SVC(C=0.1, kernel='rbf',gamma=0.1)
estimator_SVM_CFS.fit(x_train[:,CFS_fea_sele],y_train)
CFS_SVM_result=estimator_SVM_CFS.predict(x_test[:,CFS_fea_sele])

In [51]:
performan_CFS_SVM=meansure_performance(CFS_SVM_result)
print(performan_CFS_SVM)

(0.5764705882352941, 0.3561643835616438, 0.7422680412371134, 0.5098039215686274, 0.4193548387096774)


# sequencial-backward-elimination SVM

In [52]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class back_SVM:
    def __init__(self,X_train,Y_train,c,gamma):
        self.X_train=X_train
        self.Y_train=Y_train
        self.C=c
        self.Gamma=gamma
    def SBE(self,folds):
        fea_num=self.X_train.shape[1]
        candi_fea=[i for i in range(fea_num)]
        fea=[i for i in range(fea_num)]
        best_subset=[i for i in range(fea_num)]
        model=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        Best_accuracy=mean(cross_val_score(model,self.X_train,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        Success=True
        while Success:
            Success=False
            for f in candi_fea:
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(f)
                x_sele=self.X_train[:,candi_fea_copy]
                n_scores=cross_val_score(model,x_sele,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1)
                acc=mean(n_scores)
                if acc>=Best_accuracy:
                    #print(f)
                    best_subset=candi_fea_copy
                    Success=True
                    Best_accuracy=acc
            print(1)
            if Success==True:
                candi_fea=best_subset
            print(len(candi_fea))
        return best_subset
                

In [53]:
estimator_SVM_back=back_SVM(x_train,y_train,0.1,0.1)
candi_fea_SBE=estimator_SVM_back.SBE(5)

1
48
1
47
1
46
1
45
1
44
1
43
1
42
1
41
1
40
1
39
1
38
1
37
1
36
1
35
1
34
1
33
1
32
1
31
1
30
1
29
1
28
1
27
1
26
1
25
1
24
1
23
1
22
1
21
1
20
1
19
1
18
1
17
1
16
1
15
1
15


In [54]:
np.array(techini_indicator)[candi_fea_SBE],len(np.array(techini_indicator)[candi_fea_SBE])

(array(['PSY', 'MOBV', 'WVAD', 'TRIX', 'BIAS', 'CCI', 'DBCD', 'DPO', 'K',
        'D', 'J', 'RSI', 'VROC', 'WR', 'ADTM'], dtype='<U8'),
 15)

In [55]:
estimator_SVM_SBE=SVC(C=0.1, kernel='rbf',gamma=0.1)
estimator_SVM_SBE.fit(x_train[:,candi_fea_SBE],y_train)
SBE_SVM_result=estimator_SVM_SBE.predict(x_test[:,candi_fea_SBE])
performan_SBE_SVM=meansure_performance(SBE_SVM_result)
print(performan_SBE_SVM)

(0.7235294117647059, 0.547945205479452, 0.8556701030927835, 0.7407407407407407, 0.6299212598425197)


# reliefF-svm 

In [56]:
from skfeature.function.similarity_based import reliefF
relief_fea_score=reliefF.reliefF(x_train,y_train)
relief_candi_fea=list(reliefF.feature_ranking(relief_fea_score))[:9]

In [57]:
relief_fea_score

array([ 6.31742642e-02,  3.53545475e-02, -1.07830087e-02, -2.75779036e-02,
       -1.04339933e-02, -4.98403809e-03, -7.28659969e-03, -1.66241711e-02,
       -3.36336971e-03,  2.88363834e-03, -6.10250436e-03, -1.07062687e-02,
        5.20536903e-01, -1.20610674e-02, -6.05838802e-03, -9.57780373e-03,
       -9.07001200e-03, -9.85018445e-03, -1.09655459e-02,  1.36337343e-02,
       -1.13508896e-02, -4.79766747e-03,  2.99015125e-02, -1.49188814e-02,
       -1.40760734e-02,  2.83505580e-02, -1.11986433e-02,  5.05709281e-04,
       -5.86357976e-03,  6.51013221e-02, -2.33187379e-03,  2.38348686e-02,
        7.40348404e-02,  5.33687331e-02, -5.47741846e-03, -5.13988962e-03,
       -2.41534790e-02, -1.69292185e-02, -2.70530551e-02, -7.17275881e-03,
       -1.70466456e-02, -1.29316892e-02, -3.94279106e-02,  2.33300857e-03,
       -1.35287876e-02, -1.37344148e-02, -1.44196045e-02, -1.47670956e-02,
       -2.06754562e-02])

In [58]:
np.array(techini_indicator)[relief_candi_fea],len(np.array(techini_indicator)[relief_candi_fea])

(array(['MOBV', 'VRSI', 'RSI', 'QRR', 'WR', 'TAPI', 'BIAS', 'DPO', 'VROC'],
       dtype='<U8'),
 9)

In [59]:
estimator_SVM_relief=SVC(C=0.1, kernel='rbf',gamma=0.1)
estimator_SVM_relief.fit(x_train[:,relief_candi_fea],y_train)
relief_SVM_result=estimator_SVM_relief.predict(x_test[:,relief_candi_fea])
performan_relief_SVM=meansure_performance(relief_SVM_result)
print(performan_relief_SVM)

(0.7352941176470589, 0.6164383561643836, 0.8247422680412371, 0.7258064516129032, 0.6666666666666667)


# RFE-SVM

In [60]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,folds):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
   
    def featu_subset(self,featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        while True:
            if len(candi_fea)==featur_num:
                return candi_fea
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                #print(scor_fea)
                rfe_score[k]=scor_fea
                k=k+1     
            #print(rfe_score)
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            print(1)

In [61]:
rfe_SVM_model=rfe_SVM(x_train,y_train,0.1,0.1)
rfe_SVM_sele_featu_subset=rfe_SVM_model.featu_subset(9)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [62]:
np.array(techini_indicator)[rfe_SVM_sele_featu_subset],len(np.array(techini_indicator)[rfe_SVM_sele_featu_subset])

(array(['QRR', 'VOSC', 'VMACD', 'VR', 'WAD', 'MOBV', 'DPTB', 'MICD',
        'RCCD'], dtype='<U8'),
 9)

In [63]:
estimator_SVM_rfe=SVC(C=0.1, kernel='rbf',gamma=0.1)
estimator_SVM_rfe.fit(x_train[:,rfe_SVM_sele_featu_subset],y_train)
rfe_SVM_result=estimator_SVM_rfe.predict(x_test[:,rfe_SVM_sele_featu_subset])
performan_rfe_SVM=meansure_performance(rfe_SVM_result)
print(performan_rfe_SVM)

(0.7176470588235294, 0.7123287671232876, 0.7216494845360825, 0.6582278481012658, 0.6842105263157894)


# SVM-RFE_RELIEF

In [64]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_relief_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,alpha,folds):
        #kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,x_train,y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,x_train,y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            #print(relief_score)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
    def sele_feature(self,alpha,optim_featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        while True:
            if len(candi_fea)==optim_featur_num:
                return candi_fea
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            print(i)
            i-=1
            

In [65]:
# k=4# k=5, 0.8
rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1,0.1)
rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.95,9)

49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10


In [66]:
np.array(techini_indicator)[rfe_relief_SVM_sele_featu_subset],len(np.array(techini_indicator)[rfe_relief_SVM_sele_featu_subset])

(array(['QRR', 'VOSC', 'VR', 'WAD', 'MOBV', 'DPTB', 'MICD', 'RC', 'RCCD'],
       dtype='<U8'),
 9)

In [67]:
estimator_SVM_rfe_reliefF=SVC(C=0.1, kernel='rbf',gamma=0.1)
estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
print(performan_rfe_reliefF_SVM)

(0.7470588235294118, 0.7123287671232876, 0.7731958762886598, 0.7027027027027027, 0.7074829931972789)
